## Super resolution

In [1]:
%matplotlib inline

In [2]:
import fastai
from fastai.vision import *
from fastai.callbacks import *
from fastai.utils.mem import *

from torchvision.models import vgg16_bn

Untar the data at `URLs.PETS` and save the pth to `path`. Set `path/'images'` to `path_hr`, `path/'small-96'` to `path_lr`, and `path/'small-256'` to path_mr.

Create an image list from the images at `path_hr`.

Define a funtion `resize_one` that does the following:
- takes args fn, i, path, size
- gets the path `path/fn.relative_to(path_r)` and saves it to dest. what does this do?
- makes the path directory if it doesn't already exist (see path.parent.mkdir for a hint)
- Opens fn with PIL
- targ_sz = resize_to(img, size, use_min=True) -- can you explain what this does?
- create a new image `img` that resizes `img` to `targ_sz` with PIL.IMage.BILINEAR resampling, and convert it to RGB
- save the image to dest

Create a list of 2-tuples each with two elements: a path and a size. The list will containe two tuples: 
- (path_lr, 96)
- (path_mr, 256)

Iterating over them, if the path doesn't exist, call `partial(resize_one, path=p, size=size)` over `il.items` in parallel. 

Set bs to 32 and size to 128. Set rch to models.resnet34. Create an ImageItemList from the folder `path_lr` split by a random 10% with a random seed of 42.

Write a function `get_data` that takes a `bs` and a `size` and outputs a databunch. Hints:
- Label with a function that points to the file's counterpart in `path_hr`
- transform the data using `get_transforms`, only overriding `max_zoom=2.`, size=size, tfm_y=True
- normalize the `y`'s as well
- Set `data.c` to 3

Call `get_data` and assign to `data`

Show a batch of data from the validation set over 2 rows with figsize (9,9).

## Feature loss

Get the `data` of the label value of the first item in the validation set, and assign it to `t`. Then `torch.stack` it with itself. What does this do?

write a function `gram_matrix` that:
- reads `x.size()` into `n,c,h,w`. What do each of these represent?
- sets `x` to `x.view(n, c, -1)`. What does this do?
- returns x times its own transpose divided by the scalar c * h * w, which is the total number of elements in each example

Show the gram matrix of `t`.

Set base_loss to `F.l1_loss`.

Set `vgg_m` to `vgg16_bn(True).features.cuda().eval().` What does all this do? Call `requires_grad(vgg_m, False)`. What is this about?

Write a function that gets a list of the indices before every `nn.MaxPool2d` layer. Print the list of indices and the layers themselves (Hint: for the latter, you may want to use a list comprehension).

Now we're going to create our own `FeatureLoss` architecture in pytorch! First, create the scaffolding for a class `FeatureLoss` that inherits from `nn.Module`. The `__init__` method is going to take `m_feat`, `layer_ids`, and `layer_wgts`. `m_feat` is a collection of layers, like `vgg_m`. `layer_ids` is a list of ids that we'll use to index into `vgg_m`. In `__init__`, we'll want to save all of the layers from `m_feat` with indices in `layer_ids` to `self.loss_features`. Call `hook_outputs` on `self.loss_features` with `detach=False`. What does this do? Set `self.wgts` to `layer_wgts`. Set `self.metric_names` to `['pixel',]` + a list with each element being `'feat_{i}'` for each of the layer ids, and the same for `'gram_{i}'` which will represent the gram matrix metrics for each of the layer ids (more on this in a bit).

Write a method `make_features` that takes `x` and `clone=False`. It will call `self.m_feat` on `x`, which will update all the hooks in place automatically. If clone, make a clone of each of the output activations in the hooks, otherwise return the actual output activations. 

Finally, we'll write `forward`, the main method for defining a custom network in a `nn.Module`. This will take `input` and `output`. Set `out_feat` with the result of `self.make_features(target)`. Set `in_feat` to `self.make_features(input)`. Set `feat_losses` to a single-element list containing `base_loss` computed on `input` and `output`. Add to that an array containing the `base_loss` called on `f_in` and `f_out`, times the weight, for each `f_in`, `f_out`, and `weight` in `in_feat`, `out_feat`, and `weights` zipped together. Next, do the same, but the array will be `base_loss(gram_matrix(f_in), gram_matrix(f_out))*w**2*5e-3` for the `f_in`, `f_out` and `self.wgts` zipped together. Finally, we'll set `self.metrics` to a dict of `self.metric_names` zipped to `self.feat_losses`, and then return the sum of the feature losses.

Next we define `__del__`, which removes the hooks from the output activations. 

Now we instantiate a `FeatureLoss` with `vgg_m`, `blocks[2:5]`, and `[5,15,2]`. Can you explain what these args are referring to?

## Train

Set `wd=1e-3`. Create a `unet_learner` with `data`, `arch`, `wd`, `feat_loss` as our loss function, `LossMetrics` as our loss function, with `blur` set, and norm_type set to `NormType.Weight`. Can you explain what the last two do? Call `gc.collect`.

Next we run an `lr_find` and plot the results.

The lr should be somewhere around 1e-3. Set it accordingly.

Write a function `do_fit` that takes a `save_name`, a set of `lrs` that defaults to `slice(lr)`, and a `pct_start` that defaults to 0.9. It should:
- Fit a 10-epoch cycle with `lrs` and `pct_start`
- Save the results to `save_name`
- Show the results in one row with an img size of 5

Call `do_fit` with name `1a` and `slice(lr*10)`.

Unfreeze `learn`.

Call `do_fit` again with `1b` and `slice(1e-5)`.

Call `get_data` with `12` and `size*2` and set it to `data`.

Set `learn.data` to `data`, freeze it again, and `gc.collect` again.

Load `1b`.

`do_fit` on `'2a'`.

Unfreeze `learn`.

do_fit on `2b`, with `slice(1e-6, 1e-4)`, `pct_start=0.3`.

## Test

Set `learn` to `None`.

Assign `free` to the result of `gpu_mem_get_free_no_cache()`. If free > 8000, set `size` to `(1280, 1600)`, else `(820, 1024)`. Print how much you're using and how much free RAM you have. 

Create a `unet_learner` called `learn` with `data`, `arch`, `F.l1_loss`, blur, and `norm_type=NormType.Weight`.

Create `data_mr` using the datablock API. Some details:
- You'll want to create an ImageImageList from a folder, split by a random 10% with seed 42
- You'll want to label from a function that returns the image's name in `path_hr`
- You'll want to transform it with `get_transforms` and `size=size` and transform `y` as well.
- Create a databunch with a bs of 1 and normalize with imagenet_stats, and do_y as well
- Set the number of categories to 3.

Load `2b` with `learn`.

Set the data to data_mr.

Get `fn` (for filename) from the first item in the `x` part of the validation dataset.

Get `img` by opening `fn`. Print its shape.

Unpack `learn.predict(img)` into `p, img_hr, b`. What do these represent?

Show `img` with a figsize of (18, 15) and an interpolation of `nearest`.